In [2]:
!pip install ipynb

In [3]:
! pip install -U tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 6.4 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.3.0
    Uninstalling tensorflow-datasets-4.3.0:
      Successfully uninstalled tensorflow-datasets-4.3.0


In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [5]:
import sys
sys.path.insert(0,"../usr/lib")

In [6]:
import datetime
import gc
from pathlib import Path

import keras_tuner as kt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from ipynb.fs.defs.data_pipeline import EXPORTED as data_pipeline
from tensorflow import keras

In [ ]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

del gpus

# Loading builders for datasets

In [7]:
RECIPES5K_DIR = "../input/foodnet/tfrecord/recipes5k/1.0.0"
FOOD101_DIR = "../input/foodnet/tfrecord/food101/1.0.1"
NUTRITION5K_DIR = "../input/foodnet/tfrecord/nutrition5k/1.0.0"


recipes5k_food101_builder = tfds.builder_from_directories([RECIPES5K_DIR, FOOD101_DIR])
nutrition5k_builder = tfds.builder_from_directory(NUTRITION5K_DIR)

In [8]:
recipes5k_food101_builder.info.splits["train"].num_examples

105826

In [9]:
nutrition5k_builder.info.splits["train"].num_examples

271407

In [10]:
def encode_category_ingredients(category_tensor, ingredients_tensor):
    category = str(category_tensor.numpy(), "utf-8")
    ingredients = str(ingredients_tensor.numpy(), "utf-8")
    one_hot_category_tensor = (
        data_pipeline.one_hot_encoder.get_category_one_hot_encoding(category)
    )
    one_hot_ingredients_tensor = (
        data_pipeline.one_hot_encoder.get_ingredients_one_hot_encoding(
            ingredients.split(",")
        )
    )
    return (
        tf.constant(one_hot_category_tensor, dtype=tf.uint8),
        tf.constant(one_hot_ingredients_tensor, dtype=tf.uint8),
    )

In [11]:
def parse_function(x):
    encoded = tf.py_function(
        encode_category_ingredients,
        [x["category"], x["ingredients"]],
        [tf.uint8, tf.uint8],
    )
    return x["image_raw"], {
        "category_output": encoded[0],
        "calorie_output": x["calorie"],
        "carbs_output": x["carbs"],
        "protein_output": x["protein"],
        "fat_output": x["fat"],
        "ingredients_output": encoded[1],
    }

## Creating dataset for training category, ingredients and nutrients

In [12]:
# FULL DATASET
(
    recipes5k_food101_train,
    recipes5k_food101_validation,
    recipes5k_food101_test,
) = recipes5k_food101_builder.as_dataset(
    split=["train[:80%]", "train[80%:]", "train[95%:]"], shuffle_files=True
)

# # Introduce the generic category, to avoid a bias dataset, only 1k images (average image per category) is taken from nutrition5k dataset
(
    nutrition5k_category_train,
    nutrition5k_category_val,
    nutrition5k_category_test,
) = nutrition5k_builder.as_dataset(
    split=["train[:800]", "train[800:1000]", "train[950:1000]"], shuffle_files=True
)

train_dataset = recipes5k_food101_train.concatenate(nutrition5k_category_train)
validation_dataset = recipes5k_food101_validation.concatenate(nutrition5k_category_val)
test_dataset = recipes5k_food101_test.concatenate(nutrition5k_category_test)

In [15]:
# SMALLER DATASET (SUBSET OF FULL) USE THIS FOR QUICK TESTING
# (
#     recipes5k_food101_train,
#     recipes5k_food101_validation,
#     recipes5k_food101_test,
# ) = recipes5k_food101_builder.as_dataset(
#     split=["train[:10%]+train[30%:40%]", "train[50%:51%]+train[66%:70%]", "train[95%:]"],
#     shuffle_files=True,
# )

# Introduce the generic category, to avoid a bias dataset, only 1k images (average image per category) is taken from nutrition5k dataset
# (
#     nutrition5k_category_train,
#     nutrition5k_category_val,
#     nutrition5k_category_test,
# ) = nutrition5k_builder.as_dataset(
#     split=["train[:200]", "train[500:600]", "train[950:1000]"], shuffle_files=True
# )

# train_dataset = recipes5k_food101_train.concatenate(nutrition5k_category_train)
# validation_dataset = recipes5k_food101_validation.concatenate(nutrition5k_category_val)
# test_dataset = recipes5k_food101_test.concatenate(nutrition5k_category_test)

In [13]:
print(f"Total training size : {train_dataset.cardinality().numpy()}")
print(f"Total validation size : {validation_dataset.cardinality().numpy()}")
print(f"Total testing size : {test_dataset.cardinality().numpy()}")

Total training size : 85461
Total validation size : 21365
Total testing size : 5341


In [14]:
BATCH_SIZE = 32
train_dataset = (
    train_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
validation_dataset = (
    validation_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
test_dataset = (
    test_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

## Creating dataset for training ingredients and nutrients only

In [15]:
# FULL DATASET
# no_category_train, no_category_val, no_category_test = nutrition5k_builder.as_dataset(
#     split=["train[:80%]", "train[80%:98%]", "train[98%:]"], shuffle_files=True
# )

In [ ]:
# SMALLER DATASET (SUBSET OF FULL) USE THIS FOR QUICK TESTING
no_category_train, no_category_val, no_category_test = nutrition5k_builder.as_dataset(
    split=["train[:50%]", "train[80%:98%]", "train[98%:]"], shuffle_files=True
)

In [ ]:
print(f"Total training size : {no_category_train.cardinality().numpy()}")
print(f"Total validation size : {no_category_val.cardinality().numpy()}")
print(f"Total testing size : {no_category_test.cardinality().numpy()}")

In [ ]:
no_category_train = (
    no_category_train.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
no_category_val = (
    no_category_val.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
no_category_test = (
    no_category_test.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

In [16]:
gc.collect()

46

# Model Development

In [17]:
class BaseModel:
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        name,
        model_config_name,
    ):
        self.input_shape = input_shape
        self.total_food_category = total_food_category
        self.total_ingredients_category = total_ingredients_category
        self.model = None
        self.name = name
        self.model_config_name = model_config_name

    def get_input_layer(self):
        return keras.Input(shape=self.input_shape)

    def get_augmentation_layers(self):
        input_layer = keras.layers.Input(shape=self.input_shape)
        augmentation_layer = keras.layers.RandomFlip()(input_layer)
        augmentation_layer = keras.layers.RandomRotation(0.2)(augmentation_layer)
        return keras.Model(
            inputs=input_layer, outputs=augmentation_layer, name="augmentation_layers"
        )

    def get_preprocess_layers(self, input_tensor):
        raise NotImplementedError

    def get_convolution_block(self):
        raise NotImplementedError

    def get_shared_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        shared_layer = keras.layers.Flatten()(input_layer)
        shared_layer = keras.layers.Dense(
            num_units[0], activation="relu", name="shared_dense_1"
        )(shared_layer)
        shared_layer = keras.layers.BatchNormalization()(shared_layer)
#         shared_layer = keras.layers.Dense(
#             num_units[1], activation="relu", name="shared_dense_2"
#         )(shared_layer)
#         shared_layer = keras.layers.BatchNormalization()(shared_layer)
#         shared_layer = keras.layers.Dense(
#             num_units[2], activation="relu", name="shared_dense_3"
#         )(shared_layer)
#         shared_layer = keras.layers.BatchNormalization()(shared_layer)
        output_layer = keras.layers.Dropout(0.2)(shared_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="shared_layers"
        )

    def get_category_classification_layers(
        self, input_tensor, total_categories, *num_units
    ):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(
            num_units[0], activation="relu", name="category_dense_1"
        )(input_layer)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(
            num_units[1], activation="relu", name="category_dense_2"
        )(x)
        x = keras.layers.BatchNormalization()(x)
        category_classification_layer = keras.layers.Dense(
            total_categories, activation="softmax", name="category_output_layer"
        )(x)
        return keras.Model(
            inputs=input_layer,
            outputs=category_classification_layer,
            name="category_output",
        )

    def get_shared_nutrition_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        shared_nutrition_regression_layers = keras.layers.Dense(
            num_units[0], activation="relu", name="nutrition_dense_1"
        )(input_layer)
        output_layer = keras.layers.BatchNormalization()(
            shared_nutrition_regression_layers
        )
        return keras.Model(
            inputs=input_layer,
            outputs=output_layer,
            name="nutrition_regression_shared_layers",
        )

    def get_calorie_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="calorie_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="calorie_dense_2")(
            x
        )
        x = keras.layers.BatchNormalization()(x)
        calorie_regression_layers = keras.layers.Dense(1, name="calorie_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=calorie_regression_layers, name="calorie_output"
        )

    def get_carbs_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="carbs_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
#         x = keras.layers.Dense(num_units[1], activation="relu", name="carbs_dense_2")(
#             x
#         )
#         x = keras.layers.BatchNormalization()(x)
        carbs_regression_layers = keras.layers.Dense(1, name="carbs_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=carbs_regression_layers, name="carbs_output"
        )

    def get_protein_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="protein_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        protein_regression_layers = keras.layers.Dense(1, name="protein_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=protein_regression_layers, name="protein_output"
        )

    def get_fat_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="fat_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        fat_regression_layers = keras.layers.Dense(1, name="fat_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=fat_regression_layers, name="fat_output"
        )

    def get_ingredients_multilabel_layers(
        self, input_tensor, total_ingredients, *num_units
    ):
        raise NotImplementedError

    def _build(
        self,
        independent_category_units,
        independent_ingredients_units,
        independent_protein_units,
        independent_calorie_units,
        independent_carbs_units,
        independent_fat_units,
        shared_nutrition_units,
        shared_units,
    ):
        raise NotImplementedError

    def build_and_compile(self, optimizer, **kwargs):
        # Evaluation metrics and loss
        category_classification_loss = keras.losses.CategoricalCrossentropy()
        calorie_regression_loss = keras.losses.MeanAbsoluteError()
        carbs_regression_loss = keras.losses.MeanAbsoluteError()
        protein_regression_loss = keras.losses.MeanAbsoluteError()
        fat_regression_loss = keras.losses.MeanAbsoluteError()
        ingredient_multilabel_loss = keras.losses.BinaryCrossentropy()
        category_classification_metrics = (
            [
                keras.metrics.CategoricalAccuracy(name="acc"),
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall"),
                tfa.metrics.F1Score(
                    num_classes=len(data_pipeline.one_hot_encoder.all_food_categories),
                    average="macro",
                    name="macro_f1",
                ),
                tfa.metrics.F1Score(
                    num_classes=len(data_pipeline.one_hot_encoder.all_food_categories),
                    average="micro",
                    name="micro_f1",
                ),
            ],
        )

        calorie_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        carbs_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        protein_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        fat_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        ingredient_multilabel_metrics = (
            [
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall"),
                tfa.metrics.F1Score(
                    num_classes=len(data_pipeline.one_hot_encoder.all_ingredients),
                    average="macro",
                    name="macro_f1",
                ),
                tfa.metrics.F1Score(
                    num_classes=len(data_pipeline.one_hot_encoder.all_ingredients),
                    average="micro",
                    name="micro_f1",
                ),
            ],
        )
        category_classification_loss_weights = 1.0
        ingredient_multilabel_loss_weights = 1.0
        calorie_regression_loss_weights = 1.0
        carbs_regression_loss_weights = 1.0
        protein_regression_loss_weights = 1.0
        fat_regression_loss_weights = 1.0

        model = self._build(**kwargs)
        model.compile(
            optimizer=optimizer,
            loss={
                "category_output": category_classification_loss,
                "calorie_output": calorie_regression_loss,
                "carbs_output": carbs_regression_loss,
                "protein_output": protein_regression_loss,
                "fat_output": fat_regression_loss,
                "ingredients_output": ingredient_multilabel_loss,
            },
            metrics={
                "category_output": category_classification_metrics,
                "calorie_output": calorie_regression_metrics,
                "carbs_output": carbs_regression_metrics,
                "protein_output": protein_regression_metrics,
                "fat_output": fat_regression_metrics,
                "ingredients_output": ingredient_multilabel_metrics,
            },
            loss_weights={
                "category_output": category_classification_loss_weights,
                "calorie_output": calorie_regression_loss_weights,
                "carbs_output": carbs_regression_loss_weights,
                "protein_output": protein_regression_loss_weights,
                "fat_output": fat_regression_loss_weights,
                "ingredients_output": ingredient_multilabel_loss_weights,
            },
        )
        self.model = model
        return model

    def print_summary(self, verbose=True, include_convolution=False):
        assert (
            self.model is not None
        ), "Please run build_and_compile before printing summary."
        convolution_layers = set(["MobilenetV3large", "efficientnetv2-s"])
        if verbose:
            for layer in self.model.layers:
                if "input" in layer.name:
                    continue
                elif layer.name in convolution_layers and include_convolution:
                    print(layer.summary())
                elif layer.name not in convolution_layers:
                    print(layer.summary())
                else:
                    print(layer.name)
                print()
        self.model.summary()

    def build_keras_tuner(self, hp):
        hp_learning_rate = hp.Float("learning_rate", 1e-5, 1e-3, sampling="log")
        hp_optimizer = hp.Choice("optimizer", values=["Adam", "Nadam", "SGD"])
        if hp_optimizer == "Adam":
            chosen_optimizer = keras.optimizers.Adam(hp_learning_rate)
        elif hp_optimizer == "Nadam":
            chosen_optimizer = keras.optimizers.Nadam(hp_learning_rate)
        else:
            chosen_optimizer = keras.optimizers.SGD(
                hp_learning_rate, momentum=0.6, nesterov=True
            )
        hp_shared_units_1 = hp.Choice("num_units_shared_dense_1", [512, 1024, 2048])
        hp_independent_nutrition_units_1 = hp.Choice(
            "num_units_shared_nutrition_dense_1", [128, 256, 512]
        )
        hp_independent_ingredients_units_1 = hp.Choice(
            "num_units_shared_ingredients_dense_1", [256, 512, 1024]
        )
        return self.build_and_compile(
            optimizer=chosen_optimizer,
            shared_units=[hp_shared_units_1],
            independent_category_units=[],
            shared_nutrition_units=[hp_independent_nutrition_units_1],
            independent_ingredients_units=[hp_independent_ingredients_units_1],
        )

    def save_model(self, path):
        assert self.model is not None
        self.model.save(path, save_format="h5")

    def load_model(self, path):
        self.model = keras.models.load_model(path)
        return self.model

    def train_model(self, enforce_backup_empty=True, **kwargs):
        assert (
            self.model is not None
        ), "No model found. Please call build_and_compile() or load_model() before training."
        backup_dir = Path(f"./temp/backup/{self.name}/{self.model_config_name}")
        if enforce_backup_empty and backup_dir.exists():
            assert (
                len(list(backup_dir.iterdir())) == 0
            ), f"WARNING : THERE IS A BACKUP FILE FOR THIS MODEL : {self.name}."

        return self.model.fit(**kwargs, callbacks=self.get_callbacks(for_tuning=False))

    def get_callbacks(self, for_tuning):
        if for_tuning:
            tensorboard_dir = f"./models/{self.name}/tuner_logs/"
        else:
            tensorboard_dir = f"./models/logs/{self.model_config_name}/{self.name}"
            backup_restore_dir = f"./temp/backup/{self.name}/{self.model_config_name}"
#             backup_restore_callback = keras.callbacks.BackupAndRestore(
#                 backup_dir=backup_restore_dir
#             )
            checkpoint_dir = f"./temp/checkpoint/{self.name}/{self.model_config_name}"
            checkpoint_file = checkpoint_dir + "/{epoch:03d}-{val_loss:.4f}.hdf5"
            checkpoint_callback = keras.callbacks.ModelCheckpoint(
                checkpoint_file, monitor="val_loss", save_best_only=True, mode="min"
            )

        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_dir)
        early_stopping_callback = keras.callbacks.EarlyStopping("val_loss", patience=5)

        if for_tuning:
            return [tensorboard_callback, early_stopping_callback]
        else:
            return [
                tensorboard_callback,
                early_stopping_callback,
#                 backup_restore_callback,
                checkpoint_callback,
            ]

## Flat Model

In [18]:
class FlatModel(BaseModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        name,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            name,
            model_config_name,
        )

    def get_ingredients_multilabel_layers(
        self, input_tensor, total_ingredients, *num_units
    ):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[0], activation="relu", name="ingredients_dense_1"
        )(input_layer)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[1], activation="relu", name="ingredients_dense_2"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[2], activation="relu", name="ingredients_dense_3"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        output_layers = keras.layers.Dense(
            total_ingredients, activation="sigmoid", name="ingredients_output_layer"
        )(ingredients_multilabel_layers)
        return keras.Model(
            inputs=input_layer, outputs=output_layers, name="ingredients_output"
        )

    def _build(
        self,
        shared_units,
        independent_category_units,
        shared_nutrition_units,
        independent_protein_units,
        independent_fat_units,
        independent_carbs_units,
        independent_calorie_units,
        independent_ingredients_units,
    ):
        model_inputs = self.get_input_layer()
        prev_layer = self.get_augmentation_layers()(model_inputs)
        prev_layer = self.get_preprocess_layers(prev_layer)(prev_layer)
        prev_layer = self.get_convolution_block()(prev_layer)
        prev_layer = self.get_shared_layers(prev_layer, *shared_units)(prev_layer)
        category_classification_head = self.get_category_classification_layers(
            prev_layer, self.total_food_category, *independent_category_units
        )(prev_layer)
        ingredients_multilabel_head = self.get_ingredients_multilabel_layers(
            prev_layer, self.total_ingredients_category, *independent_ingredients_units
        )(prev_layer)
        # nutrition_regression_head = self.get_shared_nutrition_layers(
        #     prev_layer, *shared_nutrition_units
        # )(prev_layer)
        calorie_regression_head = self.get_calorie_regression_layers(
            prev_layer, *independent_calorie_units
        )(prev_layer)
        carbs_regression_head = self.get_carbs_regression_layers(
            prev_layer, *independent_carbs_units
        )(prev_layer)
        protein_regression_head = self.get_protein_regression_layers(
            prev_layer, *independent_protein_units
        )(prev_layer)
        fat_regression_head = self.get_fat_regression_layers(
            prev_layer, *independent_fat_units
        )(prev_layer)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        return model

### MobileNetv3

In [19]:
class FlatMobileNetv3Model(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_mobilenetv3",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.mobilenet_v3.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        mobilenet_v3_convolution_layers = keras.applications.MobileNetV3Large(
            input_shape=self.input_shape,
            include_top=False,
            weights="imagenet",
            pooling="avg",
        )
        mobilenet_v3_convolution_layers.trainable = False
        return mobilenet_v3_convolution_layers

In [20]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="1-1024-(2-256-128)-[(2-4-4)-(1-8)-(1-32)-(1-4)]-(3-1024-512-256)-FULL",
)
flat_mobilenetv3.build_and_compile(
    optimizer=keras.optimizers.Adam(),
    shared_units=[1024],
    independent_category_units=[256,128],
    shared_nutrition_units=[],
    independent_ingredients_units=[1024,512,256],
    independent_protein_units=[32],
    independent_fat_units=[4],
    independent_carbs_units=[8],
    independent_calorie_units=[4,4],
)
flat_mobilenetv3.print_summary()

17620992/17605208 [==============================] - 1s 0us/step
Model: "augmentation_layers"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
random_flip (RandomFlip)     (None, 224, 224, 3)       0         
_________________________________________________________________
random_rotation (RandomRotat (None, 224, 224, 3)       0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
None

Model: "preprocessing_layers"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
________________________

In [21]:
flat_mobilenetv3.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=100, verbose=1
)

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/100
2658/2671 [============================>.] - ETA: 2s - loss: 3.4193 - category_output_loss: 2.3256 - ingredients_output_loss: 0.0492 - calorie_output_loss: 0.7822 - carbs_output_loss: 0.1017 - protein_output_loss: 0.0679 - fat_output_loss: 0.0928 - category_output_acc: 0.4287 - category_output_precision: 0.7667 - category_output_recall: 0.2526 - category_output_macro_f1: 0.4186 - category_output_micro_f1: 0.4287 - ingredients_output_precision: 0.1626 - ingredients_output_recall: 0.1574 - ingredients_output_macro_f1: 0.0057 - ingredients_output_micro_f1: 0.1382 - calorie_output_MAE: 0.7822 - carbs_output_MAE: 0.1017 - protein_output_MAE: 0.0679 - fat_output_MAE: 0.0928

Cleanup called...


2671/2671 [==============================] - ETA: 0s - loss: 3.4426 - category_output_loss: 2.3456 - ingredients_output_loss: 0.0491 - calorie_output_loss: 0.7855 - carbs_output_loss: 0.1017 - protein_output_loss: 0.0678 - fat_output_loss: 0.0929 - category_output_acc: 0.4267 - category_output_precision: 0.7655 - category_output_recall: 0.2514 - category_output_macro_f1: 0.4177 - category_output_micro_f1: 0.4267 - ingredients_output_precision: 0.1626 - ingredients_output_recall: 0.1572 - ingredients_output_macro_f1: 0.0061 - ingredients_output_micro_f1: 0.1380 - calorie_output_MAE: 0.7855 - carbs_output_MAE: 0.1017 - protein_output_MAE: 0.0678 - fat_output_MAE: 0.0929

Cleanup called...


...

2671/2671 [==============================] - 569s 213ms/step - loss: 1.8750 - category_output_loss: 1.1415 - ingredients_output_loss: 0.0239 - calorie_output_loss: 0.5586 - carbs_output_loss: 0.0704 - protein_output_loss: 0.0214 - fat_output_loss: 0.0590 - category_output_acc: 0.6820 - category_output_precision: 0.8288 - category_output_recall: 0.5798 - category_output_macro_f1: 0.6798 - category_output_micro_f1: 0.6820 - ingredients_output_precision: 0.8183 - ingredients_output_recall: 0.3018 - ingredients_output_macro_f1: 0.0197 - ingredients_output_micro_f1: 0.1696 - calorie_output_MAE: 0.5586 - carbs_output_MAE: 0.0704 - protein_output_MAE: 0.0214 - fat_output_MAE: 0.0590 - val_loss: 2.4831 - val_category_output_loss: 1.6709 - val_ingredients_output_loss: 0.0264 - val_calorie_output_loss: 0.6067 - val_carbs_output_loss: 0.0782 - val_protein_output_loss: 0.0236 - val_fat_output_loss: 0.0774 - val_category_output_acc: 0.5735 - val_category_output_precision: 0.7499 - val_category_outp

Cleanup called...


2671/2671 [==============================] - ETA: 0s - loss: 1.8460 - category_output_loss: 1.1151 - ingredients_output_loss: 0.0238 - calorie_output_loss: 0.5564 - carbs_output_loss: 0.0704 - protein_output_loss: 0.0214 - fat_output_loss: 0.0589 - category_output_acc: 0.6879 - category_output_precision: 0.8309 - category_output_recall: 0.5905 - category_output_macro_f1: 0.6858 - category_output_micro_f1: 0.6879 - ingredients_output_precision: 0.8199 - ingredients_output_recall: 0.3042 - ingredients_output_macro_f1: 0.0199 - ingredients_output_micro_f1: 0.1706 - calorie_output_MAE: 0.5564 - carbs_output_MAE: 0.0704 - protein_output_MAE: 0.0214 - fat_output_MAE: 0.0589

Cleanup called...


2671/2671 [==============================] - 573s 214ms/step - loss: 1.8460 - category_output_loss: 1.1151 - ingredients_output_loss: 0.0238 - calorie_output_loss: 0.5564 - carbs_output_loss: 0.0704 - protein_output_loss: 0.0214 - fat_output_loss: 0.0589 - category_output_acc: 0.6879 - category_output_precision: 0.8309 - category_output_recall: 0.5905 - category_output_macro_f1: 0.6858 - category_output_micro_f1: 0.6879 - ingredients_output_precision: 0.8199 - ingredients_output_recall: 0.3042 - ingredients_output_macro_f1: 0.0199 - ingredients_output_micro_f1: 0.1706 - calorie_output_MAE: 0.5564 - carbs_output_MAE: 0.0704 - protein_output_MAE: 0.0214 - fat_output_MAE: 0.0589 - val_loss: 2.4975 - val_category_output_loss: 1.7053 - val_ingredients_output_loss: 0.0262 - val_calorie_output_loss: 0.5874 - val_carbs_output_loss: 0.0767 - val_protein_output_loss: 0.0253 - val_fat_output_loss: 0.0766 - val_category_output_acc: 0.5634 - val_category_output_precision: 0.7260 - val_category_outp

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_mobilenetv3.build_and_compile(optimizer=keras.optimizers.Adam(),num_shared_units=1024,num_shared_nutrition_units=0,num_shared_ingredients_units=0)
flat_mobilenetv3.print_summary()

In [ ]:
flat_mobilenetv3.train_model(x=train_dataset, validation_data=validation_dataset, epochs=15, verbose=1)

#### Keras Tuner

In [ ]:
tuner = kt.Hyperband(
    hypermodel=flat_mobilenetv3.build_keras_tuner,
    objective="val_loss",
    executions_per_trial=2,
    overwrite=False,
    directory="./temp/tuner",
    project_name=flat_mobilenetv3.name,
)

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(
    x=train_dataset,
    validation_data=validation_dataset,
    callbacks=flat_mobilenetv3.get_callbacks(for_tuning=True),
)

## Region Wise Model

In [ ]:
class RegionWiseModel(BaseModel):
    def __init__(
        self, input_shape, total_food_category, total_ingredients_category, name
    ):
        super().__init__(
            input_shape, total_food_category, total_ingredients_category, name
        )

    def get_ingredients_multilabel_layers(self, previous_layer, total_ingredients):
        # get the shape of the feature map (batch_size,height,width,channels)
        feature_map_shape = previous_layer.output_shape[1:]
        feature_map_height = feature_map_shape[0]
        feature_map_width = feature_map_shape[1]

        input_layer = keras.layers.Input(feature_map_shape)

        # crop the feature map into grids of feature_map_height * feature_map_width
        region_branches = []
        for row in range(1, feature_map_height + 1):
            top_crop = row - 1
            bottom_crop = feature_map_height - row
            for col in range(1, feature_map_width + 1):
                left_crop = col - 1
                right_crop = feature_map_width - col
                crop_layer = keras.layers.Cropping2D(
                    cropping=((top_crop, bottom_crop), (left_crop, right_crop))
                )(input_layer)
                flatten_layer = keras.layers.Flatten()(crop_layer)
                ingredient_prediction = keras.layers.Dense(
                    total_ingredients, activation="softmax"
                )(flatten_layer)
                reshape_layer = keras.layers.Reshape((1, total_ingredients))(
                    ingredient_prediction
                )  # reshape for global pooling 1D
                region_branches.append(reshape_layer)
        concatenate_layer = keras.layers.Concatenate(axis=1)(region_branches)
        global_max_pooling_layer = keras.layers.GlobalMaxPool1D()(concatenate_layer)
        return keras.Model(
            input_layer, global_max_pooling_layer, name="ingredients_output"
        )

    def _build(self):
        model_inputs = self.input_layer
        x = self.augmentation_layers(model_inputs)
        x = self.preprocess_layers(x)
        x = self.convolution_block(x)
        ingredients_multilabel_head = self.ingredients_multilabel_layers(x)
        x = self.shared_layers(x)
        category_classification_head = self.category_classification_layers(x)
        nutrition_regression_head = self.shared_nutrition_regression_layers(x)
        calorie_regression_head = self.calorie_regression_layers(
            nutrition_regression_head
        )
        carbs_regression_head = self.carbs_regression_layers(nutrition_regression_head)
        protein_regression_head = self.protein_regression_layers(
            nutrition_regression_head
        )
        fat_regression_head = self.fat_regression_layers(nutrition_regression_head)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        return model

### MobileNetv3 Model

In [ ]:
class RegionWiseMobileNetv3Model(RegionWiseModel):
    def __init__(self, input_shape, total_food_category, total_ingredients_category):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "RegionWiseFoodNet_with_MobileNetv3",
        )
        self._init_layers()

    def get_preprocess_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        output_layer = keras.applications.mobilenet_v3.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        mobilenet_v3_convolution_layers = keras.applications.MobileNetV3Large(
            input_shape=self.input_shape, include_top=False, weights="imagenet"
        )
        mobilenet_v3_convolution_layers.trainable = False
        return mobilenet_v3_convolution_layers

In [ ]:
regionwise_mobilenetv3 = RegionWiseMobileNetv3Model(
    (224, 224, 3),
    len(data_pipeline.one_hot_encoder.all_food_categories),
    len(data_pipeline.one_hot_encoder.all_ingredients),
)
regionwise_mobilenetv3.build_and_compile()
regionwise_mobilenetv3.print_summary()

In [ ]:
keras.utils.plot_model(
    regionwise_mobilenetv3.model,
    "./models/regionwise_foodnet_mobilenetv3.png",
    show_shapes=True,
    expand_nested=False,
)

In [ ]:
regionwise_mobilenetv3.train_model(
    x=train_dataset, epochs=10, verbose=1, validation_data=validation_dataset
)

In [ ]:
class RegionWiseEfficientNetV2Model(RegionWiseModel):
    def __init__(self, input_shape, total_food_category, total_ingredients_category):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "RegionWiseFoodNet_with_EfficientNetv2-Small",
        )
        self._init_layers()

    def get_preprocess_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        output_layer = keras.applications.efficientnet_v2.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnetv2S_convolution_layers = (
            keras.applications.efficientnet_v2.EfficientNetV2S(
                input_shape=self.input_shape, include_top=False, weights="imagenet"
            )
        )
        efficientnetv2S_convolution_layers.trainable = False
        return efficientnetv2S_convolution_layers

In [ ]:
regionwise_efficientnetv2_small = RegionWiseEfficientNetV2Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
regionwise_efficientnetv2_small.build_and_compile()
regionwise_efficientnetv2_small.print_summary(verbose=False)

In [ ]:
regionwise_efficientnetv2_small_history = regionwise_efficientnetv2_small.train_model(
    x=train_dataset, epochs=10, verbose=1, validation_data=validation_dataset
)

# Testing

In [ ]:
for x in test:
    print(x)

In [ ]:
tf.keras.utils.array_to_img(x[0].numpy())

#### Learning Rate : 0.001 (Tune from here)

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_mobilenetv3.build_and_compile(lr=0.001)
flat_mobilenetv3.print_summary(verbose=True)

In [ ]:
keras.utils.plot_model(
    flat_mobilenetv3.model,
    "./models/flat_foodnet_mobilenetv3.png",
    show_shapes=True,
    expand_nested=False,
)

In [ ]:
flat_mobilenetv3_history = flat_mobilenetv3.train_model(
    x=train_dataset, epochs=10, verbose=0, validation_data=validation_dataset
)

#### Learning Rate : Exponential Decay

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_mobilenetv3.build_and_compile(
    lr=tf.keras.optimizers.schedules.ExponentialDecay(
        0.1, decay_steps=1339, decay_rate=0.96, staircase=True
    )
)
flat_mobilenetv3.print_summary(verbose=False)

In [ ]:
flat_mobilenetv3_history = flat_mobilenetv3.train_model(
    x=train_dataset, epochs=5, verbose=1, validation_data=validation_dataset
)

#### Learning Rate : 0.008 (Failed)

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_mobilenetv3.build_and_compile(lr=0.008)
flat_mobilenetv3.print_summary(verbose=False)

In [ ]:
flat_mobilenetv3_history = flat_mobilenetv3.train_model(
    x=train_dataset, epochs=10, verbose=1, validation_data=validation_dataset
)

#### Learning Rate : 0.1 (Failed)

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_mobilenetv3.build_and_compile(lr=0.1)

In [ ]:
flat_mobilenetv3_history = flat_mobilenetv3.train_model(
    x=train_dataset, epochs=10, verbose=0, validation_data=validation_dataset
)

#### Learning Rate : 0.004 (Failed)

In [ ]:
class LRTensorBoard(keras.callbacks.TensorBoard):
    def __init__(self, log_dir, **kwargs):
        super().__init__(log_dir=log_dir, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs.update({"learning_rate": keras.backend.eval(self.model.optimizer.lr)})
        super().on_epoch_end(epoch, logs)

### EfficientNetv2 Model

In [ ]:
class FlatEfficientNetV2Model(FlatModel):
    def __init__(self, input_shape, total_food_category, total_ingredients_category):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "FlatFoodNet_with_EfficientNetV2-Small",
        )
        self._init_layers()

    def get_preprocess_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        output_layer = keras.applications.efficientnet_v2.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnetv2S_convolution_layers = (
            keras.applications.efficientnet_v2.EfficientNetV2S(
                input_shape=self.input_shape, include_top=False, weights="imagenet"
            )
        )
        efficientnetv2S_convolution_layers.trainable = False
        return efficientnetv2S_convolution_layers

In [ ]:
flat_efficientnetv2_small = FlatEfficientNetV2Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_efficientnetv2_small.build_and_compile()
flat_efficientnetv2_small.print_summary(verbose=False)

In [ ]:
flat_efficientnetv2_small_history = flat_efficientnetv2_small.train_model(
    x=train_dataset, epochs=5, verbose=1, validation_data=validation_dataset
)

### ResNet50 Model (On Hold)

In [ ]:
class FlatResNet50Model(FlatModel):
    def __init__(self, input_shape, total_food_category, total_ingredients_category):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "FlatFoodNet_with_ResNet50",
        )
        self.preprocess_layers = self.get_preprocess_layers(self.augmentation_layers)
        self.convolution_block = self.get_convolution_block()

    def get_preprocess_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        output_layer = keras.applications.resnet50.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        resnet50_convolution_layers = keras.applications.ResNet50(
            input_shape=self.input_shape, include_top=False, weights="imagenet"
        )
        resnet50_convolution_layers.trainable = False
        return resnet50_convolution_layers

In [ ]:
flat_resnet50 = FlatResNet50Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
flat_resnet50.build_and_compile()
flat_resnet50.print_summary()

In [ ]:
flat_resnet50_history = flat_resnet50.train_model(
    train_dataset, epochs=10, verbose=1, validation_data=validation_dataset
)

### EfficientNetv2 Model

In [ ]:
class RegionWiseEfficientNetV2Model(RegionWiseModel):
    def __init__(self, input_shape, total_food_category, total_ingredients_category):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "RegionWiseFoodNet_with_EfficientNetv2-Small",
        )
        self._init_layers()

    def get_preprocess_layers(self, previous_layer):
        input_layer = keras.layers.Input(shape=previous_layer.output_shape[1:])
        output_layer = keras.applications.efficientnet_v2.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnetv2S_convolution_layers = (
            keras.applications.efficientnet_v2.EfficientNetV2S(
                input_shape=self.input_shape, include_top=False, weights="imagenet"
            )
        )
        efficientnetv2S_convolution_layers.trainable = False
        return efficientnetv2S_convolution_layers

In [ ]:
regionwise_efficientnetv2_small = RegionWiseEfficientNetV2Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
)
regionwise_efficientnetv2_small.build_and_compile()
regionwise_efficientnetv2_small.print_summary(verbose=False)

In [ ]:
regionwise_efficientnetv2_small_history = regionwise_efficientnetv2_small.train_model(
    x=train_dataset, epochs=10, verbose=1, validation_data=validation_dataset
)

In [23]:
import os
os.chdir(r'/kaggle/working')

!tar -czf models.tar.gz ./models

from IPython.display import FileLink

FileLink(r'models.tar.gz')

/kaggle/working/models.tar.gz

In [1]:
import shutil
shutil.rmtree("./models")
shutil.rmtree("./temp")

FileNotFoundError: [Errno 2] No such file or directory: './temp'

In [2]:
from pathlib import Path
Path("./models.tar.gz").unlink()
Path("./temp.tar.gz").unlink()

In [3]:
Path("./temp").mkdir()
Path("./models").mkdir()
Path("./temp/backup").mkdir()
Path("./temp/checkpoint").mkdir()